In [1]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "3"

In [13]:
import subprocess
import footsteps
import os
import glob
from PIL import Image
import numpy as np
import tqdm
import torch

def DavisEval(model, name):
    davis_path = "data_storage/DAVIS/"
    output_path = footsteps.output_dir + name + "/"
    
    os.mkdir(output_path)
    with open(davis_path + "/ImageSets/2017/val.txt", "r") as f:
        sequences = f.readlines()
    for sequence in tqdm.tqdm(sequences):
        sequence = sequence[:-1]  # strip newline
        sequence_out_path = output_path + sequence + "/"
        sequence_img_path = davis_path + "JPEGImages/480p/" + sequence + "/"
        with open(davis_path + "Annotations/480p/" + sequence + "/00000.png", "rb") as handle:
          first_annotation = np.array(
            Image.open(handle)
          )

        os.mkdir(output_path + sequence)
        with open(sequence_img_path + "00000.jpg", "rb") as handle:
          first_image = np.array(Image.open(handle))

        prev_image = first_image
        prev_annotation = first_annotation

        for i in range(1, len(os.listdir(sequence_img_path))):
            with open(sequence_img_path + f"{i:05}.jpg", "rb") as handle:
              curr_image = np.array(Image.open(handle))
            annotation = model(
                first_image, first_annotation, prev_image, prev_annotation, curr_image
            )
            Image.fromarray(annotation).save(sequence_out_path + f"{i:05}.png")

            prev_image = curr_image
            prev_annoration = first_annotation
    
    subprocess.run(
        [
            "python",
            "davis2017-evaluation/evaluation_method.py",
            "--davis_path",
            "data_storage/DAVIS",
            "--results_path",
            output_path,
            "--task",
            "semi-supervised",
        ]
    )


In [14]:
import pytorch.training

In [18]:

import matplotlib.pyplot as plt
import torch.nn as nn
import torch.nn.functional as F
l = []

def execute_model(A, B, model, N=10):
    SIDE_LENGTH = 60
    FEATURE_LENGTH = 128
    
    A = torch.tensor(A).cuda().permute(0, 3, 1, 2).float()
    B = torch.tensor(B).cuda().permute(0, 3, 1, 2).float()
    feats_a = F.avg_pool2d(model(A), 2)
    feats_b = F.avg_pool2d(model(B), 2)

    # to (N, C, H*W)
    feats_a = feats_a.reshape(feats_a.shape[0],
        feats_a.shape[1], feats_a.shape[2] * feats_a.shape[3])
    feats_b = feats_b.reshape(feats_b.shape[0],
        feats_b.shape[1], feats_b.shape[2] * feats_b.shape[3])

    feats_a = feats_a.permute(0, 2, 1)

    #return feats_a
    cc = torch.bmm(feats_a, feats_b)

    cc = nn.functional.softmax(cc, dim=-1)


    cc = cc.reshape([N] + [SIDE_LENGTH] * 4)
    cc = np.array(cc.cpu().detach())



    import scipy.ndimage
    grid = np.array(
      [
        [
            [
                scipy.ndimage.measurements.center_of_mass(cc[k, i, j].transpose())
                for i in range(SIDE_LENGTH)
            ]
            for j in range(SIDE_LENGTH)
        ]
       for k in range(N)]
    )
    return cc, grid

class FMAPICON_model:
    def __init__(self, weights_path):
        self.inner_model = pytorch.training.tallerUNet2().cuda()
        self.inner_model.load_state_dict(torch.load(weights_path))
        
    def __call__(self, initial_frame, initial_mask, prev_frame, prev_mask, current_frame):
        crop = (initial_frame.shape[1] - initial_frame.shape[0]) // 2
        B = initial_frame[None, ::4, crop:-crop:4] / 255.   
        A = current_frame[None, ::4, crop:-crop:4] / 255.
        
        cc, grid = execute_model(A, B, self.inner_model, N=1)
        m = initial_mask[4::8, 4+crop:-crop:8]

        shitty_res = m[grid[0, :, :, 1].astype(int), grid[0, :, :, 0].astype(int)].transpose()

        up_res = np.repeat(np.repeat(shitty_res, 8, axis=0),8, axis=1)
        
        out_mask = initial_mask.copy()
        
        out_mask[:, crop:-crop] = up_res
        """
        plt.imshow(A[0])
        plt.show()
        plt.imshow(B[0])
        plt.show()
        plt.imshow(shitty_res)
        plt.show()
        plt.imshow(out_mask)
        plt.show()
        #l.append(up_res)
        
        plt.imshow(grid[0, :, :, 0].transpose())
        plt.show()
        
        l.append((cc, A, B, initial_mask))
        """
        

        return out_mask

#DavisEval(FMAPICON_model("tensorflow_/results/log_probability_2/epoch50/model_weights.tf"), "FMAPICON")
#DavisEval(FMAPICON_model("tensorflow_/results/clip.6_11/epoch51/model_weights.tf"), "FMAPICON")

In [17]:
for i in range(18, 20):
    DavisEval(FMAPICON_model(f"pytorch/results/60x60Iguess/network{i:05}.trch"), f"FMAPICON_downsample_fullres_eval{i:05}")

100%|█████████████████████████████████████████████████████████████████████████| 30/30 [09:40<00:00, 19.35s/it]


Evaluating sequences for the semi-supervised task...


100%|██████████| 30/30 [01:29<00:00,  2.97s/it]


Global results saved in results/downsampled_training/FMAPICON_downsample_fullres_eval00000/global_results-val.csv
Per-sequence results saved in results/downsampled_training/FMAPICON_downsample_fullres_eval00000/per-sequence_results-val.csv
--------------------------- Global results for val ---------------------------
 J&F-Mean  J-Mean  J-Recall  J-Decay   F-Mean  F-Recall  F-Decay
  0.05059 0.05816  0.009567 0.030767 0.043021  0.010166 0.030604

---------- Per sequence results for val ----------
            Sequence   J-Mean   F-Mean
      bike-packing_1 0.000000 0.000000
      bike-packing_2 0.000064 0.002271
         blackswan_1 0.015872 0.209976
         bmx-trees_1 0.018634 0.123905
         bmx-trees_2 0.000000 0.000000
        breakdance_1 0.000000 0.000000
             camel_1 0.213187 0.005458
    car-roundabout_1 0.241325 0.024031
        car-shadow_1 0.107566 0.000000
              cows_1 0.205231 0.032716
       dance-twirl_1 0.151399 0.000000
               dog_1 0.191651 0

100%|█████████████████████████████████████████████████████████████████████████| 30/30 [09:44<00:00, 19.49s/it]


...

100%|█████████████████████████████████████████████████████████████████████████| 30/30 [09:41<00:00, 19.37s/it]


Evaluating sequences for the semi-supervised task...


 97%|█████████▋| 29/30 [01:23<00:03,  3.17s/it]

Global results saved in results/downsampled_training/FMAPICON_downsample_fullres_eval00015/global_results-val.csv
Per-sequence results saved in results/downsampled_training/FMAPICON_downsample_fullres_eval00015/per-sequence_results-val.csv
--------------------------- Global results for val ---------------------------
 J&F-Mean   J-Mean  J-Recall  J-Decay  F-Mean  F-Recall  F-Decay
  0.26954 0.236661  0.170083 0.274854 0.30242  0.215031 0.312953

---------- Per sequence results for val ----------
            Sequence   J-Mean   F-Mean
      bike-packing_1 0.257844 0.588285
      bike-packing_2 0.429301 0.533850
         blackswan_1 0.685752 0.592269
         bmx-trees_1 0.045777 0.214722
         bmx-trees_2 0.104876 0.157948
        breakdance_1 0.258375 0.302740
             camel_1 0.479123 0.425402
    car-roundabout_1 0.714543 0.450332
        car-shadow_1 0.438555 0.331536
              cows_1 0.585167 0.496744
       dance-twirl_1 0.321599 0.420219
               dog_1 0.373276 0

100%|██████████| 30/30 [01:29<00:00,  2.99s/it]


In [20]:
DavisEval(FMAPICON_model(f"pytorch/results/60x60Iguess/network00019.trch"), f"FMAPICON_downsample_fullres_eval19")

100%|█████████████████████████████████████████████████████████████████████████| 30/30 [09:36<00:00, 19.20s/it]


Evaluating sequences for the semi-supervised task...


 97%|█████████▋| 29/30 [01:23<00:03,  3.17s/it]

Global results saved in results/downsampled_training/FMAPICON_downsample_fullres_eval19/global_results-val.csv
Per-sequence results saved in results/downsampled_training/FMAPICON_downsample_fullres_eval19/per-sequence_results-val.csv
--------------------------- Global results for val ---------------------------
 J&F-Mean  J-Mean  J-Recall  J-Decay   F-Mean  F-Recall  F-Decay
 0.278262 0.24379  0.182217 0.294031 0.312734  0.239102 0.350834

---------- Per sequence results for val ----------
            Sequence   J-Mean   F-Mean
      bike-packing_1 0.282394 0.584539
      bike-packing_2 0.444287 0.536627
         blackswan_1 0.735474 0.672725
         bmx-trees_1 0.040957 0.195990
         bmx-trees_2 0.103172 0.160110
        breakdance_1 0.280891 0.302948
             camel_1 0.515435 0.474407
    car-roundabout_1 0.732555 0.527231
        car-shadow_1 0.454188 0.356061
              cows_1 0.581963 0.509848
       dance-twirl_1 0.317613 0.415896
               dog_1 0.358116 0.26786

100%|██████████| 30/30 [01:29<00:00,  2.98s/it]


In [ ]:
model = FMAPICON_model("pytorch/results/no_crop_no_pretrain/network00011.trch").inner_model

In [ ]:
cc,A,B,m=l[11]

In [ ]:
cc, grid = execute_model(A, B, model, N=1)

In [ ]:
feats_A = model(torch.tensor(A).cuda().permute(0, 3, 1, 2).float()).detach().cpu().numpy()
feats_B = model(torch.tensor(B).cuda().permute(0, 3, 1, 2).float()).detach().cpu().numpy()

In [ ]:
plt.imshow(np.sum(feats_A[0 , :32]**2, axis=0))
plt.colorbar()

In [ ]:
plt.imshow(feats_A[0 , 18])
plt.colorbar()
plt.show()
plt.imshow(feats_B[0 , 18])
plt.colorbar()

In [ ]:
#plt.imshow(grid[0, :, :, 0].transpose())
#plt.show()
plt.imshow(A[0])
plt.show()
plt.imshow(B[0])
plt.show()

In [ ]:
shitty_res = m[grid[0, :, :, 1].astype(int) + 15, grid[0, :, :, 0].astype(int) + 15].transpose()

up_res = np.repeat(np.repeat(shitty_res, 4, axis=0),4, axis=1)

In [ ]:
m.shape

In [ ]:
plt.imshow(m[15:-15, 15:-15])

In [ ]:
len(l)

In [ ]:
plt.imshow(cc[0, 100, 110])
plt.colorbar()

In [21]:
9

9

In [ ]:
DavisEval(FMAPICON_model("pytorch/results/deeeep_warp/network00006.trch"), "FMAPICON3")